# https://github.com/ShuaiBai623/AI-City-Anomaly-Detection/blob/master/src/bg_modeling/get_ignore_area.py

In [19]:
import skimage
from skimage.measure import label 
from scipy.ndimage.filters import gaussian_filter
import cv2 as cv
import os
import pandas as pd
import numpy as np
import re

In [2]:
def load_video(filename):
    # Loads a video with opencv, returns PIL Images for each frame
    
    vid = cv.VideoCapture(filename)

    while vid.isOpened():
        has_frame, img = vid.read()

        if has_frame:
            yield img
        else:
            break
            
    vid.release()
    
def get_video_shape(filename):
    vid = cv.VideoCapture(filename)
    _, img = vid.read()
    vid.release()
    
    return img.shape

In [13]:
def combine_boxes(fbf_bbox_df, img_height, img_width, score_threshold=0.1, normalize=True):
    count_matrix = np.zeros((img_height, img_width))
    for frame, df in fbf_bbox_df.groupby("frame"):
        tmp_score = np.zeros((img_height, img_width))

        for x1, y1, x2, y2, score in df[["x1", "y1", "x2", "y2", "score"]].values:
            x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))

            if score > score_threshold:
                tmp_score[y1:y2, x1:x2] = np.maximum(score, tmp_score[y1:y2, x1:x2])  # add all the boxes into one image

        count_matrix += tmp_score

    if normalize:
        # scale to [0, 1]
        count_matrix = (count_matrix - count_matrix.min()) / (count_matrix.max() - count_matrix.min())
    
    return count_matrix

In [4]:
def get_connected_regions(mask, area_threshold=2000,):
    regions = label(mask, connectivity = 1) # get connected regions
    
    for region_idx in np.unique(regions):
        if region_idx == 0: # 0 is background
            continue
        
        region_mask = regions == region_idx
        if region_mask.sum() < area_threshold: # get rid of small regions
            mask = np.where(region_mask, False, mask)
            
    return mask

In [15]:
def create_ignore_mask(frame_by_frame_path, ignore_matrix_path, img_shape, count_threshold=0.08, area_threshold=2000, score_threshold=0.1, gaussian_sigma=3):
    # Read in bboxes
    fbf_bbox_df = pd.read_csv(frame_by_frame_path, header=None, 
                          names=["frame", "NA", "x1", "y1", "w", "h", "score", "NA1", "NA2", "NA3"])
    fbf_bbox_df["x2"] = fbf_bbox_df["x1"] + fbf_bbox_df["w"]
    fbf_bbox_df["y2"] = fbf_bbox_df["y1"] + fbf_bbox_df["h"]
    
    # Combine bboxes
    heatmap = combine_boxes(fbf_bbox_df, img_shape[0], img_shape[1], score_threshold)
    
    # Create ignore mask
    mask = heatmap > count_threshold
    mask = get_connected_regions(mask, area_threshold) 
    mask = gaussian_filter(mask.astype(float), gaussian_sigma) > count_threshold
    
    # Save ignore mask
    np.save(ignore_matrix_path, mask)
    

In [39]:
def process_folder(frame_by_frame_folder, video_folder, ignore_matrix_folder):
    """
    Both video and frame by frame results must exist for each case.
    """
    
    for fbf_filename in sorted(os.listdir(frame_by_frame_folder)):
        match = re.match("^video(\d+)\.txt$", fbf_filename) # video123.txt
        if not match:
            continue
            
        video_id = int(match.groups()[0])
        video_path = os.path.join(video_folder, f"{video_id}.mp4")
        frame_by_frame_path = os.path.join(frame_by_frame_folder, fbf_filename)
        ignore_matrix_path = os.path.join(ignore_matrix_folder, f"{video_id}.npy")
        
        print(f"Processing video {video_id}...")
        
        img_shape = get_video_shape(video_path)
        create_ignore_mask(frame_by_frame_path, ignore_matrix_path, img_shape)

---

In [6]:
frame_by_frame_folder = "/data/aicity/winner_team/detection_results/test_framebyframe"
video_folder = "/data/aicity/test/"
ignore_matrix_folder = "/data/aicity/winner_team/detection_results/test_seg_masks"

In [38]:
process_folder(frame_by_frame_folder, video_folder, ignore_matrix_folder)

Processing video 2...
